In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import sklearn as sk
import pyodbc
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import sys
from string import Template
from csv import QUOTE_ALL 
import time
import os 

import teradata

import json
import requests
import requests.auth
import sys

"""

Standard Metrics API Caller. Provides the following metrics: 
      "traffic_in",
      "traffic_out",
      "dwell_count",
      "avg_dwell_duration",
      "engagement_rate"

Outputs: results_<startdate>_<enddate>.csv

To use the RetailNext API, please refer to http://docs.retailnext.net/cloud/api/ for information. 

Step 1: Create Tokens at https://<subscription>.cloud.retailnext.net/admin/system-api-tokens
Step 2: Input Tokens into the file below
Step 3: Change the start date and end date below
Step 4: Press shift enter to run

If you would like to change any of the queries, please use change the post_request variable below.

Implementation Notes:

There were two approaches I could have done for locations. First approach was to iterate through each 
location for eaach query.
The second approach was to embed all locations inside the initial request file. 
I chose approach 1. 


Work to do: 
Better Pagination handling. 
More Error Handling  
"""




""" PLACE YOUR ACCESS CODES HERE """
ACCESS_CODE = "********"
SECRET_KEY =  "********"
CUSTOMER = "*****"
GET_URL = "https://" + CUSTOMER + ".api.retailnext.net/v1/location"
POST_URL = "https://" + CUSTOMER + ".api.retailnext.net/v1/datamine"


import datetime

conn=pyodbc.connect('DRIVER={Teradata Database ODBC Driver 16.20};DBCNAME=*****;UID=*****;PWD=*****;QUIETMODE=YES;')

max_date_plus_one = pd.read_sql("select max(txn_strt_dt)+1  from ABFRL_VRDM.SLS_TXN_LINE", con=conn)
max_date_plus_one=str(max_date_plus_one.iloc[0,0])

START_DATE = '2017-09-01'
#START_DATE=max_date
STOP_DATE=str(pd.to_datetime('today'))[:10]

OUT_FILE = "./AEO_Traffic_" + START_DATE + "_" + STOP_DATE + ".csv"
f = open(OUT_FILE, 'w')
f.close()

""" Get all the locations and print them """
def get_locations():

    # Initial Call
    request = requests.get(GET_URL, auth=(ACCESS_CODE, SECRET_KEY))
    request.encoding = 'ISO-8859-1'
    j = request.json()
   
    for location in j['locations']:
        get_metrics(location) #Pass the location off to the call for metadata later. 

    #Handle Pagination 
    while(isNextPage(request)): #Partial 
          print('Working on Next Page', request.headers['X-Page-Next'])
          headers = {'X-Page-Start': request.headers['X-Page-Next']}
          request = requests.get(GET_URL, auth=(ACCESS_CODE, SECRET_KEY), headers=headers)
          j = request.json()

          for location in j['locations']:
             get_metrics(location) #Pass the location off to the call for metadata later.


"""Check if status code is 206. If 206, then pagination is occuring. Need to iterate."""
def isNextPage(request):
    if(request.status_code == 206):
      return True
    return False

""" TODO: Generic Function to support Pagination Requests. Takes a input call and adds headers to it. """
def getNextPage(request):
    headers = {'X-Page-Start': request.headers['X-Page-Next']}


""" Get all available metrics. Test the post call. """
def get_metrics(location):
    location_id = location['id']
    if location['type'] == "store":
      print("Getting Metrics for location:  ", location_id)
      post_request["locations"][0] = location_id
      payload = json.dumps(post_request, ensure_ascii=False)
      post = requests.post(POST_URL, auth=(ACCESS_CODE, SECRET_KEY), data = payload , headers={'content-type': 'application/json'}
  )
      try: #in case the data that comes back can't be converted to json. 
        j = post.json()
        writeToCSV(j, location)
      except ValueError:
        print('JSON value failed to return for this location')

      while(isNextPage(post)): #Partial 
          print("Getting Metrics for location:  ", location_id)
          headers = {'content-type': 'application/json', 'X-Page-Start': request.headers['X-Page-Next']}
          payload = json.dumps(post_request, ensure_ascii=False)
          post = requests.post(POST_URL, auth=(ACCESS_CODE, SECRET_KEY), data = payload , headers=headers)
          try:
              j = post.json()
              writeToCSV(j, location)
          except ValueError:
              print('JSON value failed to return for this location')
      
   
"""
   Right now this is an hourly pull from the data, 
   assuming structured data form. At some point, it would be nice to 
   Adjust this to handle unstructured data and alternative queries. 
"""
def writeToCSV(data, location_info):
    with open(OUT_FILE, "a") as file:
        for metrics in data['metrics']:
            for data in metrics['data']:
                    line = location_info['store_id'] + "," \
                    + str(data['group']['finish']) +  "," \
                    + str(data['value'])+"\n" 
                    file.write(line)
                    

def Final_inserting():
    df_=pd.read_csv(OUT_FILE,header=None)
    udaExec = teradata.UdaExec (appName="test", version="1.0", logConsole=False)
    with udaExec.connect(method="odbc",system="", username="",
                      password="", driver="Teradata Database ODBC Driver 16.20") as connect:
        data = [tuple(x) for x in df_.to_records(index=False)]
        connect.executemany("INSERT INTO ABFRL_TSTAGE1_DEV.AEO_Traffic values(?,?,?)",data,batch=True)
    print("Records Inserted Succesfully")
        







post_request = {
   "locations" : [
      "<location-here>"
   ],
   "time_ranges" : [
      {
         "type" : "store_hours"
      }
   ],
   "group_bys" : [
      {  "value": 1, 
         "unit": "days",
         "group": "date"
      } ,
   ],
   "date_ranges" : [
      {
         "first_day" : START_DATE,
         "last_day" : STOP_DATE
      }
   ],
   "metrics" : [
      "traffic_in",
   ]
}

def main():
  get_locations()
  Final_inserting()
  


if __name__ == "__main__":
  main()


